### Create dataset from Hugging Face and write to UC

In [ ]:
UC_CATALOG = "users"
UC_SCHEMA = "alex_miller"
UC_TABLE = "imdb"

In [0]:
from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

In [0]:
from pyspark.sql import functions as F
import pandas as pd

train_dataset = ds['train'].to_pandas()
val_dataset = ds['unsupervised'].to_pandas()
test_dataset = ds['test'].to_pandas()
all_dataset = pd.concat([train_dataset, val_dataset, test_dataset], ignore_index=True)
spark_dataframe = spark.createDataFrame(all_dataset) \
    .withColumn("id", F.monotonically_increasing_id())

display(spark_dataframe)

In [0]:
spark_dataframe.write.mode("overwrite").saveAsTable(f"{UC_CATALOG}.{UC_SCHEMA}.{UC_TABLE}")

### Create Embeddings using AI_QUERY

In [ ]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {UC_CATALOG}.{UC_SCHEMA}.{UC_TABLE}_embeddings AS
          SELECT
            *,
            AI_QUERY(
              'databricks-gte-large-en', 
              text
            ) AS embeddings
          FROM {UC_CATALOG}.{UC_SCHEMA}.{UC_TABLE}""")

In [ ]:
spark.table(f"{UC_CATALOG}.{UC_SCHEMA}.{UC_TABLE}_embeddings").display()